# Working with APIs in Python

Making API requests in Python can be really simple. There's a low-level module called urllib that can also make the kinds of web requests that we want, but it's not as friendly as the `requests` module, which we'll be using.

In [1]:
import requests

## Authentication

You'll have to authenticate each request to the Harvard Art Museum API with an API key. Other APIs may require different kinds of authentication (sometimes very complicated auth! Look for libraries at that point), but HAM has some pretty simple authentication, which makes things easy for us. You can sign up for a key [here](https://www.harvardartmuseums.org/collections/api).

In [2]:
APIKEY = "" # Enter your API key here

## Basic request

We're going to start off with a basic request to the API. This API, like many others, has a variety of endpoints, each with their own url, slightly modified from a base url. We'll worry about the general case in a bit, for now let's look at a basic API request.

In this example, we'll re-create the first example in the [Object endpoint documentation](https://github.com/harvardartmuseums/api-docs/blob/master/object.md), which will give each of you the records for 10 objects that have never been viewed online in the museum's collections.

In [3]:
url = "https://api.harvardartmuseums.org/object"
parameters = {
    "q":"totalpageviews:0",
    "size":10,
    "apikey":APIKEY
}
R = requests.get(url,params=parameters)
R.json()

{'info': {'next': 'https://api.harvardartmuseums.org/object?q=totalpageviews%3A0&size=10&apikey=43d8d250-0c20-11e8-83ec-17c87ad2c768&page=2',
  'page': 1,
  'pages': 5624,
  'totalrecords': 56231,
  'totalrecordsperquery': 10},
 'records': [{'accessionmethod': 'Gift',
   'accessionyear': 1984,
   'accesslevel': 1,
   'century': None,
   'classification': 'Plaques',
   'classificationid': 60,
   'colorcount': 0,
   'commentary': None,
   'contact': 'am_asianmediterranean@harvard.edu',
   'contextualtextcount': 0,
   'copyright': None,
   'creditline': 'Harvard Art Museums/Arthur M. Sackler Museum, Gift of Mrs. Beatrice Kelekian in memory of her husband, Charles Dikran Kelekian',
   'culture': 'Near Eastern',
   'datebegin': -2000,
   'dated': 'c. 2000-1000 BCE',
   'dateend': -1000,
   'dateoffirstpageview': '2018-01-18',
   'dateoflastpageview': '2018-01-21',
   'department': 'Department of Ancient and Byzantine Art & Numismatics',
   'description': 'Molded plaque with six pointed rays

### Refresher on Dictionaries

Python dictionaries are sets of key / value pairs, where a value can be accessed by its key. You're essentially naming a value in a container, so you can easily call it up later.

Dictionaries have very fast lookups, so you can get a value from its key very quickly, no matter how large the dictionary is. However, they are also unordered, so if you iterate through all of the key / value pairs in the dictionary, there's no guarantee that they'll be in the same order.

We're just going to be looking up data in dictionaries, so here's a quick refresher on the syntax:

In [4]:
parameters['q']

'totalpageviews:0'

In [5]:
parameters['apikey'] # This also works when we've set the value to another variable

'43d8d250-0c20-11e8-83ec-17c87ad2c768'

In [6]:
parameters['q'] = "totalpageviews:1" # You can also set the value of a key like you would a variable

## Making a Request

The request syntax is so simple, you might have missed it. Let's query again for objects with only one pageview, and take a closer look.

In [7]:
R = requests.get(url,params=parameters)

### Formatted parameters

That request has created a request object, which contains not only the data that we get from the Harvard Art Museums, but information on the request we sent, like the URL that it used. Notice that requests has turned our query parameter dictionary into a GET request at the end of our URL.

If you've been working with API requests or web scraping before, you might be used to seeing URLs get constructed like this:

```python
url = "https://api.harvardartmuseums.org/object?q=" + query + "&apikey=" + apikey
```

If you have, I'm sure you'll appreciate how much simpler this is, especially when dealing with more query parameters.

In [8]:
R.url

'https://api.harvardartmuseums.org/object?q=totalpageviews%3A1&size=10&apikey=43d8d250-0c20-11e8-83ec-17c87ad2c768'

### Taking a look at the results

Request objects have a built-in method, `.json()`, which converts a JSON file received as a response to a request from a string of text that happens to be in this data format into Python native data structures, like lists, dictionaries, numbers and strings. We can use this method to see a dictionary representation of what we've gotten from the API request.

In [9]:
R.json()

{'info': {'next': 'https://api.harvardartmuseums.org/object?q=totalpageviews%3A1&size=10&apikey=43d8d250-0c20-11e8-83ec-17c87ad2c768&page=2',
  'page': 1,
  'pages': 2396,
  'totalrecords': 23960,
  'totalrecordsperquery': 10},
 'records': [{'accessionmethod': 'Transfer',
   'accessionyear': 2011,
   'accesslevel': 1,
   'century': '20th century',
   'classification': 'Photographs',
   'classificationid': 17,
   'colorcount': 0,
   'commentary': None,
   'contact': 'am_moderncontemporary@harvard.edu',
   'contextualtextcount': 0,
   'copyright': None,
   'creditline': 'Harvard Art Museums/Fogg Museum, Transfer from the Carpenter Center for the Visual Arts, American Professional Photographers Collection',
   'culture': 'American',
   'datebegin': 1945,
   'dated': 'c. 1950',
   'dateend': 1955,
   'dateoffirstpageview': '2013-02-07',
   'dateoflastpageview': '2013-02-07',
   'department': 'Department of Photographs',
   'description': None,
   'dimensions': 'image: 12.7 x 10.16 cm (5 x 

## Changing our request

Let's say we're not interested in the most obscure parts of the collection (pot sherds, apparently), but rather in the most popular parts of the collection. There are a few ways we might go about doing this. One way might be to sort our search results by `totalpageviews`, and see what the top 10 are.

To do that, we can go back to the [API documentation](https://github.com/harvardartmuseums/api-docs/blob/master/object.md) and look for hints about what we might be able to do.

In [10]:
parameters = {
    "size":10,
    "apikey":APIKEY,
    "sort": "totalpageviews",
    "sortorder": "desc"
}
R = requests.get(url,params=parameters)
R.json()

{'info': {'next': 'https://api.harvardartmuseums.org/object?size=10&apikey=43d8d250-0c20-11e8-83ec-17c87ad2c768&sort=totalpageviews&sortorder=desc&page=2',
  'page': 1,
  'pages': 23295,
  'totalrecords': 232947,
  'totalrecordsperquery': 10},
 'records': [{'accessionmethod': 'Bequest',
   'accessionyear': 1951,
   'accesslevel': 1,
   'century': '19th century',
   'classification': 'Paintings',
   'classificationid': 26,
   'colorcount': 10,
   'colors': [{'color': '#64af7d',
     'css3': '#5f9ea0',
     'hue': 'Green',
     'percent': 0.2979781420765,
     'spectrum': '#4fb94f'},
    {'color': '#64c896',
     'css3': '#66cdaa',
     'hue': 'Green',
     'percent': 0.21289617486339,
     'spectrum': '#47b853'},
    {'color': '#323219',
     'css3': '#2f4f4f',
     'hue': 'Brown',
     'percent': 0.19814207650273,
     'spectrum': '#3db657'},
    {'color': '#7d7d4b',
     'css3': '#696969',
     'hue': 'Green',
     'percent': 0.056775956284153,
     'spectrum': '#6cbd45'},
    {'color

### Looking at the results

Often, you'll want to look at some specific aspect of the data you're getting. Since the API returns everything, you'll have to format the output in some friendly, readable format. In this next cell, we'll iterate through the results and print them  out in a nicer format.

We're being pretty low-level with the text formatting here, and one important key to understanding this bit is that "\t" means "tab", so that you can insert that character, which normally does something else.

Feel free to play around with this cell to format it more to your liking.

In [11]:
records = R.json()['records']
print("views\tartwork")
print()
for record in records:
    print("{}\t{}".format(record['totalpageviews'],record['title']))
    # `.format` puts its arguments sequentially in the string calling it wherever there are {} pairs
    # It does a lot more than that, with more advanced documentation here: 
    # https://docs.python.org/3.4/library/string.html#id1
    print()

views	artwork

23782	Self-Portrait Dedicated to Paul Gauguin

18497	The Gare Saint-Lazare: Arrival of a Train

13978	Bahram Gur Fights the Horned Wolf (painting, verso; text, recto), illustrated folio from a manuscript of the Great Ilkhanid Shahnama (Book of Kings)

12875	Odalisque with a Slave

11821	A Mother and Child and Four Studies of Her Right Hand, 1904; verso:  Self-Portrait Standing, 1903

10834	Jeanne-Antoinette Poisson, Marquise de Pompadour

9160	Red Boats, Argenteuil

8900	Court of Gayumars (painting, recto; text, verso), folio from a manuscript of the Shahnama by Firdawsi

7256	Self-Portrait in Tuxedo

6967	Mother and Child



The top result from this query is a Van Gogh painted titled "Self-Portrait Dedicated to Paul Gauguin." You can grab just the first object by accessing the records list (which is indexed from 0):

In [12]:
topResult = R.json()['records'][0]
topResult

{'accessionmethod': 'Bequest',
 'accessionyear': 1951,
 'accesslevel': 1,
 'century': '19th century',
 'classification': 'Paintings',
 'classificationid': 26,
 'colorcount': 10,
 'colors': [{'color': '#64af7d',
   'css3': '#5f9ea0',
   'hue': 'Green',
   'percent': 0.2979781420765,
   'spectrum': '#4fb94f'},
  {'color': '#64c896',
   'css3': '#66cdaa',
   'hue': 'Green',
   'percent': 0.21289617486339,
   'spectrum': '#47b853'},
  {'color': '#323219',
   'css3': '#2f4f4f',
   'hue': 'Brown',
   'percent': 0.19814207650273,
   'spectrum': '#3db657'},
  {'color': '#7d7d4b',
   'css3': '#696969',
   'hue': 'Green',
   'percent': 0.056775956284153,
   'spectrum': '#6cbd45'},
  {'color': '#969664',
   'css3': '#808080',
   'hue': 'Green',
   'percent': 0.043715846994536,
   'spectrum': '#84c441'},
  {'color': '#afaf7d',
   'css3': '#bdb76b',
   'hue': 'Green',
   'percent': 0.042622950819672,
   'spectrum': '#9ecb3b'},
  {'color': '#4b9664',
   'css3': '#2e8b57',
   'hue': 'Green',
   'perc

You can easily access properties from the image record:

In [13]:
topResult['title']

'Self-Portrait Dedicated to Paul Gauguin'

Try adding an additional code field to the notebook below to access information about Van Gogh. If that's easy, try displaying all HAM works by Van Gogh and filtering to only records with an image associated.

## More endpoints to love

You might notice, looking at the documentation, that we've only been accessing the "objects" API endpoint, when there are many other endpoints that we could ask for information.

A note on terminology, an API endpoint is a one place that you can go to ask specific questions about a certain part of a dataset or service. Many APIs, especially commercial APIs, contain many, many endpoints, to facilitate all sorts of different activity on a platform.

For example, you can take a look at the [reddit API documentation](https://www.reddit.com/dev/api/) (which we won't be using, this is just an example), to see all of the different endpoints that an application might need to serve as an alternative front end for reddit. 

Endpoints on the same API are likely to behave similarly, but they will all serve different purposes. Looking at our HAM endpoints, it looks like they all follow the same basic formulation: https://api.harvardartmuseums.org/RESOURCE_TYPE . We can use this to our advantage, and create a function to query any endpoint easily.

In [14]:
def ham_query(apikey, endpoint, **kwargs):
    """Sends kwargs to the specified endpoint, using apikey for authentication"""
    params = kwargs
    params['apikey'] = apikey
    url = "https://api.harvardartmuseums.org/{}".format(endpoint)
    R = requests.get(url,params=params)
    return R

In [15]:
response = ham_query(APIKEY, "gallery", floor=2)

In [16]:
response.json()

{'info': {'next': 'https://api.harvardartmuseums.org/gallery?floor=2&apikey=43d8d250-0c20-11e8-83ec-17c87ad2c768&page=2',
  'page': 1,
  'pages': 3,
  'totalrecords': 24,
  'totalrecordsperquery': 10},
 'records': [{'floor': 2,
   'galleryid': 2200,
   'gallerynumber': '2200',
   'id': 2200,
   'labeltext': 'A sustained commitment to the tenets of neoclassicism persisted after the French Revolution, and artists such as Jacques-Louis David and his school continued to adhere to the sculptural and archaeological approach to form that they had helped popularize in the previous century. However, their style was soon challenged by romanticism, which proposed a radically different kind of representation. Its development over the first half of the nineteenth century produced myriad reformulations and contradictions in its definition. But at its height, romantic painting was characterized by a bold and vibrant palette and by loose and expressive brushstrokes that often obscured any careful prep

### Boy, that's convenient!

That function works because Python has this neat ability to take arbitrary arguments in functions, if you tell it to. Essentially, there are two special arguments in function definitions: `*args` and `**kwargs`. These make available `args` and `kwargs` objects, respectively, in your function. `args` is a list, and `kwargs` is a dictionary. This makes it so that you don't have to specify all of the arguments your function can take, you can just give it general rules for lists or key pairs of data as input.

You might be wondering why you wouldn't just use a dictionary or list instead of those arguments. In our case, it's mostly a stylistic choice, and one that saves us a couple of key strokes.

In [33]:
# Try out some other endpoints!
response = ham_query(APIKEY, "exhibition", status="current")
current_exhibits = response.json()['records']
current_exhibits
print()
for exhibit in current_exhibits:
    print("{} ({} to {})".format(exhibit['title'],exhibit['begindate'],exhibit['enddate'])) 


John Russell: Australia's French Impressionist (2018-07-21 to 2018-11-11)
Animal-Shaped Vessels from the Ancient World: Feasting with Gods, Heroes, and Kings (2018-09-07 to 2019-01-06)
Lorenzo Lotto Portraits / Lorenzo Lotto Retratos (2018-06-19 to 2019-02-10)
Harvey Quaytman: A Retrospective (2018-10-17 to 2019-01-27)
The Chiaroscuro Woodcut in Renaissance Italy (2018-06-03 to 2019-01-20)
Diana Extended Loan at Huntington Library (1984-07-01 to None)
Drawing in Tintoretto's Venice (2018-10-12 to 2019-05-26)
Delacroix (2018-09-17 to 2019-01-06)
The Construction of the World - Art and the Economy (2018-10-11 to 2019-02-03)
Adam and Eve (2018-09-01 to 2019-01-06)


## Individual Objects and IIIF

The HAM object API can provide more information (such as `exhibition`, `citation`, `publication`, and `marks`) if you ask for a specific object by its objectid. For some records that have been extensively annotated (often those with `verificationlevel` == 4) the lists for these properties can contain hundreds of entries.

In [34]:
objectid = topResult['objectid']
objectid
parameters = {
    "apikey": APIKEY
}
objectUrl = url + "/" + str(objectid)
R = requests.get(objectUrl, parameters)
topResultFull = R.json()
print(topResultFull['verificationlevel'] == 4)
topResultFull

True


{'accessionmethod': 'Bequest',
 'accessionyear': 1951,
 'accesslevel': 1,
 'century': '19th century',
 'classification': 'Paintings',
 'classificationid': 26,
 'colorcount': 10,
 'colors': [{'color': '#64af7d',
   'css3': '#5f9ea0',
   'hue': 'Green',
   'percent': 0.2979781420765,
   'spectrum': '#4fb94f'},
  {'color': '#64c896',
   'css3': '#66cdaa',
   'hue': 'Green',
   'percent': 0.21289617486339,
   'spectrum': '#47b853'},
  {'color': '#323219',
   'css3': '#2f4f4f',
   'hue': 'Brown',
   'percent': 0.19814207650273,
   'spectrum': '#3db657'},
  {'color': '#7d7d4b',
   'css3': '#696969',
   'hue': 'Green',
   'percent': 0.056775956284153,
   'spectrum': '#6cbd45'},
  {'color': '#969664',
   'css3': '#808080',
   'hue': 'Green',
   'percent': 0.043715846994536,
   'spectrum': '#84c441'},
  {'color': '#afaf7d',
   'css3': '#bdb76b',
   'hue': 'Green',
   'percent': 0.042622950819672,
   'spectrum': '#9ecb3b'},
  {'color': '#4b9664',
   'css3': '#2e8b57',
   'hue': 'Green',
   'perc

When we printed the 10 most popular records above (under Looking at the Results), you may have noticed a sharp dropoff after the first few records. Our Van Gogh painting is particularly popular, with ~5000 more views than the second most popular record and almost 4x as many as the tenth most popular. This particular Art Museum record is used as the default image asset for the demo installation of [Project Mirador](http://projectmirador.org/demo/), an image viewer for [IIIF (International Image Interoperability Framework)](https://iiif.io/). 

We're not going to go deep into IIIF in this workshop, but want to mention that IIIF is both a community of developers and a collection of APIs and API-compliant tools that you can use to share, manipulate, and display visual materials. The [Image API]() and [Presentation API]() are the most used outputs as of now, though there are also APIs for Authentication, Search, and beta versions for other media (video and VR).

(Explanation of Image and Presi APIs (manifest, canvas especially) - images)

Within our topResultFull object, there is an images list, which contains IIIF baseurls as well as Image Delivery Service URLs:

In [69]:
images = topResultFull['images']
images

[{'baseimageurl': 'https://nrs.harvard.edu/urn-3:HUAM:DDC251942_dynmc',
  'copyright': 'President and Fellows of Harvard College',
  'displayorder': 1,
  'format': 'image/jpeg',
  'height': 2550,
  'idsid': 47174896,
  'iiifbaseuri': 'https://ids.lib.harvard.edu/ids/iiif/47174896',
  'imageid': 429030,
  'publiccaption': None,
  'renditionnumber': 'DDC251942',
  'width': 2087},
 {'baseimageurl': 'https://nrs.harvard.edu/urn-3:HUAM:DDC000072_dynmc',
  'copyright': 'President and Fellows of Harvard College',
  'displayorder': 2,
  'format': 'image/jpeg',
  'height': 2550,
  'idsid': 18737483,
  'iiifbaseuri': 'https://ids.lib.harvard.edu/ids/iiif/18737483',
  'imageid': 185978,
  'publiccaption': None,
  'renditionnumber': 'DDC000072',
  'width': 2088},
 {'baseimageurl': 'https://nrs.harvard.edu/urn-3:HUAM:DDC251934_dynmc',
  'copyright': 'President and Fellows of Harvard College',
  'displayorder': 3,
  'format': 'image/jpeg',
  'height': 2550,
  'idsid': 47174892,
  'iiifbaseuri': 'htt

This particular record has 6 images associated with it. Try opening some of the urls:

In [78]:
for index, image in enumerate(images, start=1):
    print("image{} baseimageurl: {}\nimage{} iiifbaseuri: {}".format(index,image['baseimageurl'],index,image['iiifbaseuri']))

image1 baseimageurl: https://nrs.harvard.edu/urn-3:HUAM:DDC251942_dynmc
image1 iiifbaseuri: https://ids.lib.harvard.edu/ids/iiif/47174896
image2 baseimageurl: https://nrs.harvard.edu/urn-3:HUAM:DDC000072_dynmc
image2 iiifbaseuri: https://ids.lib.harvard.edu/ids/iiif/18737483
image3 baseimageurl: https://nrs.harvard.edu/urn-3:HUAM:DDC251934_dynmc
image3 iiifbaseuri: https://ids.lib.harvard.edu/ids/iiif/47174892
image4 baseimageurl: https://nrs.harvard.edu/urn-3:HUAM:30033_dynmc
image4 iiifbaseuri: https://ids.lib.harvard.edu/ids/iiif/43182083
image5 baseimageurl: https://nrs.harvard.edu/urn-3:HUAM:50493_dynmc
image5 iiifbaseuri: https://ids.lib.harvard.edu/ids/iiif/43183405
image6 baseimageurl: https://nrs.harvard.edu/urn-3:HUAM:50849_dynmc
image6 iiifbaseuri: https://ids.lib.harvard.edu/ids/iiif/43183422


You'll notice that the `baseimageurls` use Harvard's Name Resolution service, which redirects to an Image Delivery Service URL. We're more interested in the `iiifbaseuris` because we can manipulate IIIF resources using the Image API. Try opening one of those.

The IIIF Image API spec requires that we pass not just a baseurl, but a well-formed IIIF-compliant URI to get an image. Let's check out that [documentation](https://iiif.io/api/image/2.1/) and see what else we need to construct one of those.

From the docs:

>The IIIF Image API URI for requesting an image must conform to the following URI Template:
>
>`{scheme}://{server}{/prefix}/{identifier}/{region}/{size}/{rotation}/{quality}.{format}`
>
>For example:
>
>`http://www.example.org/image-service/abcd1234/full/full/0/default.jpg`
>
The parameters of the Image Request URI include region, size, rotation, quality and format, which define the characteristics of the returned image. These are described in detail in Image Request Parameters.

The `iiifbaseuri`s include up through the `{identifier}`, but we need to include additional parameters to get the server to actually render the image for us. These parameters are passed within the URI itself, rather than in a query string appended after a delimiter (usually `?`), which is what we've been using `requests` to do. Let's write a function that can generate IIIF URIs for us. Because all of the parameters we want to insert are required, we won't use `**kwargs` - instead we'll set default params which you can override by passing in new ones.

In [91]:
def iiif_query(baseuri, region="full", size="full", rotation=0, quality="default", format="jpg", info=False):
    """Creates a valid IIIF URL, with the option to request image information"""
    if baseuri[-1:] != "/":
        baseuri += "/"
    if info == True:
        return baseuri+"info.json"
    else:
        url = baseuri+"{}/{}/{}/{}.{}".format(region, size, rotation, quality, format)
        return url

In [95]:
for image in images:
    print(iiif_query(image['iiifbaseuri']))

https://ids.lib.harvard.edu/ids/iiif/47174896/full/full/0/default.jpg
https://ids.lib.harvard.edu/ids/iiif/18737483/full/full/0/default.jpg
https://ids.lib.harvard.edu/ids/iiif/47174892/full/full/0/default.jpg
https://ids.lib.harvard.edu/ids/iiif/43182083/full/full/0/default.jpg
https://ids.lib.harvard.edu/ids/iiif/43183405/full/full/0/default.jpg
https://ids.lib.harvard.edu/ids/iiif/43183422/full/full/0/default.jpg


Now we have some valid image URLs that you can open in your browser!

This is nice, but the Image API lets us do a lot more by just by passing in some parameters. Maybe we want to generate some square, grayscale images for a gallery:

In [97]:
for image in images:
    print(iiif_query(image['iiifbaseuri'], quality="gray", region="square"))

https://ids.lib.harvard.edu/ids/iiif/47174896/square/full/0/gray.jpg
https://ids.lib.harvard.edu/ids/iiif/18737483/square/full/0/gray.jpg
https://ids.lib.harvard.edu/ids/iiif/47174892/square/full/0/gray.jpg
https://ids.lib.harvard.edu/ids/iiif/43182083/square/full/0/gray.jpg
https://ids.lib.harvard.edu/ids/iiif/43183405/square/full/0/gray.jpg
https://ids.lib.harvard.edu/ids/iiif/43183422/square/full/0/gray.jpg


Let's try requesting only the right half of an image, in black and white, and getting back a PNG:

In [103]:
print(iiif_query([image][0]['iiifbaseuri'], quality='bitonal', region="pct:50,0,100,100", format="png"))

https://ids.lib.harvard.edu/ids/iiif/43183422/pct:50,0,100,100/full/0/bitonal.png


Feel free to try to manipulate the images in other ways as well! That's it for our quick introduction to the Image API.

Later in the session we'll consume these resources using [Mirador](http://projectmirador.org/), an image viewer which uses the IIIF Image and Presentation APIs. If you're interested in the Presi API, check out [this documentation](https://iiif.io/api/presentation/2.1/) to learn how IIIF manifests structure sequences of canvases which image viewers then present to end users. You can find an HAM Object's manifest in the `seeAlso` property, or by by appending the object ID to a baseurl:

In [112]:
print(topResult['seeAlso'])
print('https://iiif.harvardartmuseums.org/manifests/object/{}'.format(topResult['id']))

[{'id': 'https://iiif.harvardartmuseums.org/manifests/object/299843', 'type': 'IIIF Manifest', 'format': 'application/json', 'profile': 'http://iiif.io/api/presentation/2/context.json'}]
https://iiif.harvardartmuseums.org/manifests/object/299843


## More stuff!

So far, we've only been getting limited sets of object data. But what if there were a big query we wanted to make? Let's try it out on "Unidentified culture" materials in the museum.

In [48]:
unknown = ham_query(APIKEY, "object", culture="Unidentified culture", size=100).json()

Looking at our previous queries, it looks like we've got some information about our query in the "info" section. Let's take a look at that...

In [47]:
unknown['info']

{'next': 'https://api.harvardartmuseums.org/object?culture=Unidentified%20culture&size=50&apikey=43d8d250-0c20-11e8-83ec-17c87ad2c768&page=2',
 'page': 1,
 'pages': 13,
 'totalrecords': 607,
 'totalrecordsperquery': 50}

### Iterating through pages

It looks like we have 7 pages of data to get, and our response gives us a "next" url for easy iteration. Nice!

However, let's look at how we would iterate even without this convenience factor.

In [49]:
unknown.keys()

dict_keys(['info', 'records'])

It looks like we have two components to our response, info and records. Since `info` is request specific, we're just after `records`, and we'll want to combine them all. 

We could set this up in a regular loop, which would query the API as fast as our processors can go, which can produce many queries per second, and is usually limited more by network speed than by processor speed. However, this can put a strain on the API endpoint, so it can be good practice to build in timers when making many requests. Sometimes an API will specify a number of requests/second that you're allowed to make, sometimes not. Putting even a fraction of a second delay in your code will help make sure that you don't accidentally get yourself banned from the API.

In [50]:
import time

In [51]:
unknown_records = []
keepGoing = True
page = 1

while keepGoing:
    R = ham_query(APIKEY, "object", culture="Unidentified culture", size=100, page=page)
    time.sleep(0.5)
    response = R.json()
    unknown_records.extend(response['records'])
    if response['info']['pages'] == page:
        keepGoing = False
    else:
        page += 1

In [62]:
len(unknown_records)
unknown_records

[{'accessionmethod': 'Purchase',
  'accessionyear': 1982,
  'accesslevel': 1,
  'century': '19th century',
  'classification': 'Photographs',
  'classificationid': 17,
  'colorcount': 0,
  'commentary': None,
  'contact': 'am_europeanamerican@harvard.edu',
  'contextualtextcount': 0,
  'copyright': None,
  'creditline': 'Harvard Art Museums/Fogg Museum, Purchase through the generosity of Melvin R. Seiden',
  'culture': 'Unidentified culture',
  'datebegin': 1860,
  'dated': '1860-1868',
  'dateend': 1868,
  'dateoffirstpageview': '2009-11-22',
  'dateoflastpageview': '2018-09-09',
  'department': 'Department of Photographs',
  'description': None,
  'dimensions': 'photograph: 13.9 x 10.5 cm (5 1/2 x 4 1/8 in.)\r\npage: 27.1 x 17.4 cm (10 11/16 x 6 7/8 in.)',
  'division': 'European and American Art',
  'edition': None,
  'exhibitioncount': 0,
  'groupcount': 0,
  'id': 98543,
  'imagecount': 0,
  'imagepermissionlevel': 0,
  'labeltext': None,
  'lastupdate': '2018-11-05T03:42:45-0500'

## Exporting

Now we have some cool data, but maybe we want to do something with it outside of Python. It's common to see CSV data traded around, since it's just a plain text spreadsheet file, so most things can parse it. Let's make one of those! We could use the relatively low level `csv` library, but instead, let's use a higher level library, `pandas`

In [53]:
import pandas as pd # Common invocation of pandas. Gotta save those 4 keystrokes.

### "Be a dataframe!" - us

Pandas thinks of things in terms of dataframes, which will be familiar if you work in R. Basically, they're really efficient arrays of data. They also translate really well to a tabular format.

To make an iterable object into a dataframe, sometimes you can just get away with shouting "Hey you! Be a dataframe!" at it (in code). Since we have a list of dictionaries with consistent keys, there's a good chance this process will do something smart for us:

In [54]:
pd.DataFrame(unknown_records)

,accessionmethod,accessionyear,accesslevel,century,classification,classificationid,colorcount,colors,commentary,contact,...,technique,techniqueid,title,titlescount,totalpageviews,totaluniquepageviews,url,verificationlevel,verificationleveldescription,worktypes
0,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Emmanuel Leutze (1816-1868),2,24,16,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '45', 'worktype': 'book page'}..."
1,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Asher Brown Durand (c. 1796-1886),2,8,7,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '45', 'worktype': 'book page'}..."
2,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Henry Peters Gray (1819-1877),2,10,10,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."
3,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (view of house labeled Ballingeir),2,1,1,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '9', 'worktype': 'album page'}..."
4,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (field with two men standing to right...,2,0,0,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."
5,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (Auld Bridge of Dough),2,1,1,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."
6,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (view of house with nine figures in f...,2,1,1,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."
7,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (landscape with stream or river label...,2,3,2,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '9', 'worktype': 'album page'}..."
8,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (Ken Loch),2,2,2,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '9', 'worktype': 'album page'}..."
9,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Henry Inman (1801-1846),2,12,8,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."


What do you know! It worked. But let's take a look at a more hands on approach to the same thing.

In [55]:
pd.DataFrame.from_dict(unknown_records)

,accessionmethod,accessionyear,accesslevel,century,classification,classificationid,colorcount,colors,commentary,contact,...,technique,techniqueid,title,titlescount,totalpageviews,totaluniquepageviews,url,verificationlevel,verificationleveldescription,worktypes
0,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Emmanuel Leutze (1816-1868),2,24,16,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '45', 'worktype': 'book page'}..."
1,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Asher Brown Durand (c. 1796-1886),2,8,7,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '45', 'worktype': 'book page'}..."
2,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Henry Peters Gray (1819-1877),2,10,10,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."
3,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (view of house labeled Ballingeir),2,1,1,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '9', 'worktype': 'album page'}..."
4,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (field with two men standing to right...,2,0,0,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."
5,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (Auld Bridge of Dough),2,1,1,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."
6,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (view of house with nine figures in f...,2,1,1,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."
7,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (landscape with stream or river label...,2,3,2,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '9', 'worktype': 'album page'}..."
8,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Untitled (Ken Loch),2,2,2,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '9', 'worktype': 'album page'}..."
9,Purchase,1982.0,1,19th century,Photographs,17,0,NaN,None,am_europeanamerican@harvard.edu,...,Albumen silver print,110.0,Henry Inman (1801-1846),2,12,8,https://www.harvardartmuseums.org/collections/...,3,Good. Object is well described and information...,"[{'worktypeid': '259', 'worktype': 'photograph..."


`pd.DataFrame.from_dict` gives you more control over the conversion process, so you can provide more options if things don't look how you expect them to.

As a side note, we do have some data structures in here that don't make a lot of sense in a tabular format. Look at `worktypes` at the very end. That's a list, and each cell has list data in it. We won't be able to do much with that in Excel or some other tabular data processing tool, but it also won't break anything for us. It just looks weird. Within the dataframe, they still work like lists though, so you can access the data while you're still in Python if you're clever about it.

### Exporting

From here, our export process is really easy. We just say "Hey you! Be a CSV file now!", and so it shall be.

In [56]:
df = pd.DataFrame(unknown_records)
df.to_csv("unknown_ham_records.csv",index=None)

# Data collected!

Now we've got some interesting data, and we can throw it into a program like Tableau or Excel to visualize it or further explore it. We could also continue to explore it in Python, but that's a workshop for another day.

# Reverse the flow!

Now that we've got some information from the Harvard Art Museum API, let's look at how we can send that information somewhere else to add content to another site. 

Here, we'll use the API for Omeka. Omeka is a content management system, like WordPress, but focused on making the collections of libraries, archives, and museums more easily accessible on the web. It's built around the concept of items, and focuses on describing those items, collecting them sensibly, and incorporating them into online narratives.

The site we'll be using is the site that we use for testing our Omeka service here: http://testing.omeka-dev.fas.harvard.edu/

You'll find documentation for the API here: http://omeka.readthedocs.io/en/latest/Reference/api/index.html

Our goal for this portion will be to use the documentation and what we've already learned to create items in Omeka representing each of the places in our dataframe.

Before we get started on that, we'll want to see how the API represents items, so we can copy that when creating new ones.

In [116]:
omeka_api_key = 'ee94996a5f853132c15299eee3141ffc57aa34c2' # We'll give you a key to use for the site

In [123]:
# We're not using the key yet, since we're just viewing public information
R = requests.get('http://demo.omeka.fas.harvard.edu/api/items/', params={'key':omeka_api_key})
demo_item = R.json()

In [124]:
demo_item

[{'added': '2018-08-29T16:31:48+00:00',
  'collection': None,
  'element_texts': [{'element': {'id': 64,
     'name': 'UUID',
     'resource': 'elements',
     'url': 'http://demo.omeka.fas.harvard.edu/api/elements/64'},
    'element_set': {'id': 5,
     'name': 'IIIF Item Metadata',
     'resource': 'element_sets',
     'url': 'http://demo.omeka.fas.harvard.edu/api/element_sets/5'},
    'html': False,
    'text': '824010cd-20f7-4a49-a795-874ac4b82ff8'}],
  'extended_resources': {'exhibit_pages': {'count': 0,
    'resource': 'exhibit_pages',
    'url': 'http://demo.omeka.fas.harvard.edu/api/exhibit_pages?item=136'}},
  'featured': False,
  'files': {'count': 0,
   'resource': 'files',
   'url': 'http://demo.omeka.fas.harvard.edu/api/files?item=136'},
  'id': 136,
  'item_type': None,
  'modified': '2018-08-29T16:31:48+00:00',
  'owner': {'id': 15,
   'resource': 'users',
   'url': 'http://demo.omeka.fas.harvard.edu/api/users/15'},
  'public': False,
  'tags': [],
  'url': 'http://demo.

## New function!

Since making items isn't exactly intuitive, let's make a quick function to construct items from dictionaries. We're relying on some things specific to this site, namely the IDs of each element, and for a more general solution we'd want to do something more nuanced than hard coding those IDs into our workflow. For now, though, this is a workable solution

In [126]:
def make_item(element_texts):
    """
    Takes a dictionary with format {element_id:element_text, ...}
    """
    base_item = {
        'element_texts':[],
        'featured': False,
        'public': True,
    }
    for _id, text in element_texts.items():
        element = {
            'element': { 'id': int(_id) },
            'text': text,
            'html': True
        }
        base_item['element_texts'].append(element)
    return base_item

In [130]:
test = {
    50: 'A Test Item',
    41: "The description of the test item. It might be a bit longer, which is fine since it won't be used as a page title or anything."
}
test_item = make_item(test)
print(test_item)

{'element_texts': [{'element': {'id': 50}, 'text': 'A Test Item', 'html': True}, {'element': {'id': 41}, 'text': "The description of the test item. It might be a bit longer, which is fine since it won't be used as a page title or anything.", 'html': True}], 'featured': False, 'public': True}


## POST new data

Now that we have content to upload, let's take a look at how we'll do that. We're using a different method of sending data to the url, you'll notice. We're POSTing data, which usually means we're adding something new. We can still use our `params` argument, but our data is in our `json` argument.

The `requests` module has this as a convenient parameter, so you don't have to turn your dictionary into a string to use it as a data payload. Since this is such a common task, `requests` has built it into this method call so we can just use the dictionary object we've created.

We'll still get a response, but in this case, we'll get a representation of the item that we just created, as long as it was created successfully. We know this from the documentation, which tells us what response to expect from each kind of query we can send to the items API endpoint.

In [128]:
R = requests.post('http://demo.omeka.fas.harvard.edu/api/items',json=test_item, params={'key':omeka_api_key})

In [129]:
R.json()

{'added': '2018-11-06T20:14:45+00:00',
 'collection': None,
 'element_texts': [{'element': {'id': 50,
    'name': 'Title',
    'resource': 'elements',
    'url': 'http://demo.omeka.fas.harvard.edu/api/elements/50'},
   'element_set': {'id': 1,
    'name': 'Dublin Core',
    'resource': 'element_sets',
    'url': 'http://demo.omeka.fas.harvard.edu/api/element_sets/1'},
   'html': True,
   'text': 'A Test Item - New'},
  {'element': {'id': 41,
    'name': 'Description',
    'resource': 'elements',
    'url': 'http://demo.omeka.fas.harvard.edu/api/elements/41'},
   'element_set': {'id': 1,
    'name': 'Dublin Core',
    'resource': 'element_sets',
    'url': 'http://demo.omeka.fas.harvard.edu/api/element_sets/1'},
   'html': True,
   'text': "The description of the test item. It might be a bit longer, which is fine since it won't be used as a page title or anything."},
  {'element': {'id': 64,
    'name': 'UUID',
    'resource': 'elements',
    'url': 'http://demo.omeka.fas.harvard.edu/ap

## Functions in functions

We can make another function to take a dictionary that represents our item in a pretty convenient way and add that directly to Omeka. We're using the function that we made to create an item within this function, so we don't have to add that functionality to this function too. We might want to keep these functions separate, in case we want to use the `make_item` function on its own for some other purpose, like creating several items and then adding them all at once.

The second function is designed to go from a single row from our spreadsheet directly to the creation of an Item in Omeka. You can change the items that you add by changing this function. In fact, I encourage you to do so, since we'll all be adding items, and it'll be hard to tell which ones are yours. 

In [131]:
def add_item_to_omeka(element_texts):
    item = make_item(element_texts)
    print(item)
    #R = requests.post('http://demo.omeka.fas.harvard.edu/api/items',json=item, params={'key':omeka_api_key})
    #return R.json()

In [ ]:
def row_to_omeka(row):
    title = row['Name']
    description = "Located in {}, MA ({},{})".format(row['City'],row['lat'],row['lng'])
    element_texts = {
        '50': title,
        '41': description,
        '39': 'Your name here!'
    }
    response = add_item_to_omeka(**element_texts)
    return response

In [ ]:
df.apply(row_to_omeka,axis=1)